# 少数クラス

## データ読み込みと確認

In [ ]:
import pandas as pd

data = pd.read_csv("officialSource/qss-master/CAUSALITY/STAR.csv")

# データ外観
data.shape

# どこにどれくらい欠損がある？
data.isnull().sum()
# カウントする時は、classtypeをカウントすると良さそう

## 練習1
- データフレームに kinder という因子変数作成し、classtypeの整数値をわかりやすいラベルに変更
- race 変数を、アジア系、ネイティブ・アメリカンを others にまとめ、white, black, hispanic, others の4レベルの因子変数に変更。race変数に直接上書きする
- 欠損値除外をする

In [ ]:
data.loc[data["classtype"] == 1, "kinder"] = "small"
data.loc[data["classtype"] == 2, "kinder"] = "regular"
data.loc[data["classtype"] == 3, "kinder"] = "reg w/aid"

data.loc[data["race"].isin([3, 5, 6]), "race"] = "others"
data.loc[data["race"].isin([1]), "race"] = "white"
data.loc[data["race"].isin([2]), "race"] = "black"
data.loc[data["race"].isin([4]), "race"] = "hispanic"
data.groupby(by="race").count()

data_nonMissing = data[~data.isnull().any(axis=1)].copy()

## 練習2
- 4年次の「読解」と「算数」の成績について、少人数クラス所属の生徒と標準規模クラス所属の生徒とを比較する
    - 「読解」と「算数」の平均値を、「少人数クラス」「標準規模クラス」のグループ別で比較
    - 欠損レコードは除外しておく
- 結果の実質的な解釈を簡潔に述べる
- 推定された効果の大きさを理解するため、テスト点数の標準偏差を比べる

In [ ]:
temp = data.groupby(by="kinder").mean()[["g4math", "g4reading"]]
# 読解の平均の差
temp.loc["small", "g4reading"] - temp.loc["regular", "g4reading"]
# 算数の平均の差
temp.loc["small", "g4math"] - temp.loc["regular", "g4math"]

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="white", color_codes=True)

# 標準偏差を出す
temp = data.groupby(by="kinder").std()[["g4math", "g4reading"]]
temp.loc["small", "g4reading"] - temp.loc["regular", "g4reading"]
temp.loc["small", "g4math"] - temp.loc["regular", "g4math"]
# ばらつきを確認
ax = sns.distplot(data_nonMissing.loc[data_nonMissing["kinder"]=="small", "g4reading"], label="small")
sns.distplot(data_nonMissing.loc[data_nonMissing["kinder"]=="regular", "g4reading"], ax=ax, label="regular")
ax.legend()
plt.show()

In [ ]:
# 模範解答では、個別に標準偏差は出さず、全体で出していたのでやってみる
data["g4reading"].std(skipna=True)
data["g4math"].std(skipna=True)

少数クラスと、標準規模クラスでは、読解の平均点の差が3.5点、算数の平均点の差が-0.34点であった。

標準偏差の値が、読解は52.4259、算数が43.0921である。このバラツキが起きる集団において、平均点の差が4点未満というのは、
充分に偶然起こりうる範囲内の点差と考えられるので、少数クラスによる介入効果は無いと考えられる。

## 練習3
- 少人数クラスと標準規模クラスの、点数の範囲を調べる
    - 66パーセンタイルで示されるハイスコア、33パーセンタイルで示されるロースコア同士を比較する
- この分析＋練習2の平均値による分析とを合わせて、なにか言えるか？

In [ ]:
small_df = data[data["kinder"]=="small"].copy()
regular_df = data[data["kinder"]=="regular"].copy()

small_df["g4reading"].quantile(0.33) - regular_df["g4reading"].quantile(0.33)
small_df["g4reading"].quantile(0.66) - regular_df["g4reading"].quantile(0.66)
small_df["g4math"].quantile(0.33) - regular_df["g4math"].quantile(0.33)
small_df["g4math"].quantile(0.66) - regular_df["g4math"].quantile(0.66)

3分位数の差もかなり小さく、ほぼ差がない状態。これは、分布にも差がない事を示しており、前述の平均値でも差がない事から、やはり幼稚園時代のクラスの規模が与えた変化は特になかった、と考えられる。

## 練習4
- 幼稚園時代に小規模クラスにいたかどうかのフラグと、その後も小規模クラスに何年所属し続けたかのデータがあるので、クロス集計表(人数)を作る
- 少人数クラスにより長く在籍した場合、テストの点数に大きな変化はあるだろうか？平均値、中央値を見比べてみよう

In [ ]:
# クロス集計表を作る
data.groupby(by=["kinder","yearssmall"]).count()["classtype"].unstack("yearssmall").to_csv("ch2/cross.csv", encoding="utf-8-sig")

# yearssmall別で、色々と指標を見てみる
data.groupby(by="yearssmall").describe()[["g4math", "g4reading"]]

# 分布図を書いてみる g4reading
ax = sns.distplot(data.loc[data["yearssmall"] == 0, "g4reading"].dropna(), label="0")
ax = sns.distplot(data.loc[data["yearssmall"] == 1, "g4reading"].dropna(), label="1", ax=ax)
ax = sns.distplot(data.loc[data["yearssmall"] == 2, "g4reading"].dropna(), label="2", ax=ax)
ax = sns.distplot(data.loc[data["yearssmall"] == 3, "g4reading"].dropna(), label="3", ax=ax)
ax = sns.distplot(data.loc[data["yearssmall"] == 4, "g4reading"].dropna(), label="4", ax=ax)
ax.legend()
plt.show()

# 分布図を書いてみる g4math
temp = data.loc[data["yearssmall"] == 0, "g4math"]
ax = sns.distplot(temp.dropna(), label="0")
ax = sns.distplot(data.loc[data["yearssmall"] == 1, "g4math"].dropna(), label="1", ax=ax)
ax = sns.distplot(data.loc[data["yearssmall"] == 2, "g4math"].dropna(), label="2", ax=ax)
ax = sns.distplot(data.loc[data["yearssmall"] == 3, "g4math"].dropna(), label="3", ax=ax)
ax = sns.distplot(data.loc[data["yearssmall"] == 4, "g4math"].dropna(), label="4", ax=ax)
ax.legend()
plt.show()

## 練習5 「STARプロジェクトで人種間の学力差はうまったのか？」を調べる
- classtype別の、白人orマイノリティの平均点比較をする

In [ ]:
del data["race_class"]

data.loc[data["race"]=="white", "race_class"] = "white"
data.loc[data["race"].isin(["black", "hispanic"]), "race_class"] = "minority"
temp = data.groupby(by=["kinder", "race_class"]).mean().unstack("race_class")
temp

temp.loc["regular", ("g4reading", "white")] - temp.loc["regular", ("g4reading", "minority")]
temp.loc["small", ("g4reading", "white")] - temp.loc["small", ("g4reading", "minority")]
temp.loc["regular", ("g4math", "white")] - temp.loc["regular", ("g4math", "minority")]
temp.loc["small", ("g4math", "white")] - temp.loc["small", ("g4math", "minority")]
# 分布状況を把握
data.groupby(by=["kinder", "race_class"]).describe()[["g4reading"]]
data.groupby(by=["kinder", "race_class"]).describe()[["g4math"]]


# 毎度、分布を見ておく
# minority の読解点数の分布
ax = sns.distplot(data.loc[(data["race_class"] == "minority") & (data["kinder"] == "small"), "g4reading"].dropna(), label="small")
ax = sns.distplot(data.loc[(data["race_class"] == "minority") & (data["kinder"] == "regular"), "g4reading"].dropna(), label="regular", ax=ax)
ax = sns.distplot(data.loc[(data["race_class"] == "minority") & (data["kinder"] == "reg w/aid"), "g4reading"].dropna(), label="reg w/aid", ax=ax)
ax.legend()
plt.show()



読解力においては、標準クラスの白人とマイノリティの点差が36点なのに対し、小規模クラスでは28点と、8点ほど縮まっている。同様の計算をすると、算数ではほぼ変化無し。
このことから、読解力においては、マイノリティの学習効果は上がる傾向がある。

## 練習6
- 異なるタイプのクラスに割り当てられた生徒の高校卒業率を比較
- 少人数クラスに在籍した期間別の卒業率を比較
- STARプロジェクトが白人とマイノリティ(黒人とヒスパニック)の卒業率の格差を縮めたかどうかを調べる

In [ ]:
data.dropna(subset=["hsgrad"]).groupby("kinder", as_index=False).mean()
data.groupby("kinder", as_index=False)[["hsgrad"]].describe()
# 2値なので、クロス集計で分布を確認
pd.crosstab(data["kinder"],data["hsgrad"], margins=True)

# 検証・pandas の平均は、None系はそもそも対象外(＝分母に入れない)として計算してくれてる。
import numpy as np
pd.Series([1,2,3,np.nan]).mean()


data.groupby(by="yearssmall", as_index=False)[["hsgrad"]].describe()
pd.crosstab(data["yearssmall"], data["hsgrad"], margins=True)

# マイノリティ別の卒業率
# 幼稚園時のクラス規模が、人種間の卒業率格差に貢献できていたか？
temp = data.groupby(by=["race_class", "kinder"], as_index=False).mean()[["kinder", "race_class", "hsgrad"]]
temp = temp.pivot(index="kinder", columns="race_class")
temp[("hsgrad","white")] - temp[("hsgrad","minority")]

temp = data.groupby(by=["race_class", "yearssmall"], as_index=False).mean()[["yearssmall", "race_class", "hsgrad"]]
temp = temp.pivot(index="yearssmall", columns="race_class")
temp[("hsgrad","white")] - temp[("hsgrad","minority")]

高校卒業率は、幼稚園時のクラス規模別での違いは見られないが、小規模クラスにいた年数別で見てみると、卒業率は異なっている。
1年間だけ所属するよりも、4年所属していた方が8%ポイントほど卒業率が高く、且つ、所属した期間が無い人よりも5%ポイント高い。
このことから、小規模クラスに所属した年数が多いほど高校卒業率は高まり、3年以上所属すると、所属したことが無い人よりも高校卒業率が高くなる。

人種間の格差をSTARプロジェクトが貢献したかどうかについては、上記同様、幼稚園時のクラス規模は特には貢献しておらず、小規模クラス所属期間が長いほど縮まる事もなく、逆に開いていたので、人種間格差については特に貢献はしていない。

# 同性婚に関する意見

## 練習1
ベースラインインタビューを使って、ランダム化が正しく行われているかを確認する

In [ ]:
data2 = pd.read_csv("officialSource/qss-master/CAUSALITY/gay.csv")
data2.shape
data2.isnull().sum() # 欠損なし

# どういうトリートメントが行われた調査だったのかを把握するための describe
data2.groupby(["study", "treatment"]).describe()

# wave1群でのインタビューは、戸別訪問が行われる前に実施されている(＝介入前に実施)ので、
# このwave1 の値を見れば、無作為抽出されているのかどうかを確認できる。
temp = data2[(data2["wave"] == 1) & (data2["study"]==1)]
temp.groupby("treatment").describe()[["ssm"]]

ssmスコア平均がほぼ同じで、中央値やSTDもほぼ差がないので、無作為抽出できていると考えられる。

## 練習2
戸別訪問(＝介入)が行われた2ヶ月後、wave2調査が実施された。ゲイに接触した人とそうでない人との間で、同性婚支持率の違いがあったかどうかを推定して！

In [ ]:
temp = data2[(data2["wave"] == 2) & (data2["study"] == 1)]
temp.groupby("treatment").describe()[["ssm"]]
temp = temp.groupby("treatment").mean()[["ssm"]]

temp.loc["Same-Sex Marriage Script by Gay Canvasser", "ssm"] - temp.loc["No Contact", "ssm"]
temp.loc["Same-Sex Marriage Script by Straight Canvasser", "ssm"] - temp.loc["No Contact", "ssm"]

当事者接触アリの人は支持率0.10アップ、ナシの人は0.12アップという結果。
当事者接触の効果は特になさそう。

## 練習3
この調査では、なぜ同性婚支持台本を使わない個別訪問を行ったトリートメントグループも加えられているのだろうか？
- study1, wave2 を使って、「ゲイの訪問員による同性婚支持台本」と「ゲイの訪問員によるリサイクル促進台本」のトリートメント効果を比較しよう
- 同様に、ストレートの人も比較しよう
- これらの比較からわかることはなんだろうか？

In [ ]:
temp = data2[(data2["study"] == 1) & (data2["wave"]==2)]
temp = temp.groupby("treatment").mean()[["ssm"]]
temp.loc["Same-Sex Marriage Script by Gay Canvasser", "ssm"] - temp.loc["Recycling Script by Gay Canvasser", "ssm"]
temp.loc["Same-Sex Marriage Script by Straight Canvasser", "ssm"] - temp.loc["Recycling Script by Straight Canvasser", "ssm"]

当事者に接するだけで、直接的に同性婚支持を訴えずとも支持率は上がるが、(台本が違ってもどちらも同じくらいの支持率上昇貢献度)
当事者に接しない場合は、直接的に同性婚支持を訴えないと支持率上昇には貢献しない。(台本が違うと、支持率上昇貢献度も違う)

## 練習4
- 1年後のインタビュー調査から、戸別訪問の効果が持続しているかどうかを見る
    - 各waveごとに、コントロールを比較対象として、同性婚台本を用いたゲイとストレートの訪問員の平均効果を求める
    - 持続している場合は、どのような条件で持続しているのだろうか？

In [ ]:
temp = data2[data2["study"]==1].groupby(by=["treatment", "wave"]).mean()[["ssm"]].unstack("treatment")
temp[("ssm", "Same-Sex Marriage Script by Gay Canvasser")] - temp[("ssm", "No Contact")]
temp[("ssm", "Same-Sex Marriage Script by Straight Canvasser")] - temp[("ssm", "No Contact")]

訪問員がゲイの人の方が、1年後も支持率を維持しやすい

## 練習5
研究1を再現するために、研究2が行われた。
- 研究2について、「ゲイの訪問員による同性婚台本」「個別訪問なし」を使って、ランダム化が適切に行われているかを調べて！
    - ベースラインとしては wave1を使う

In [ ]:
temp = data2[data2["study"]==2]
temp[temp["wave"]==1].groupby(by="treatment").describe()["ssm"]

ssmの平均値が一致、標準偏差もほぼ同じなので、正しくランダム化出来ている

## 練習6
- 第2波のデータから、ゲイの訪問員によるトリートメント効果を推定しなはれ
- その結果は、研究1と一致しているか？

In [ ]:
temp = temp[temp["wave"]==2].groupby(by="treatment").mean()[["ssm"]]
temp.loc["Same-Sex Marriage Script by Gay Canvasser", "ssm"] - temp.loc["No Contact", "ssm"] 

ゲイの訪問員と接する場合と、訪問員ナシの差は0.12。研究1とほぼ同等。

## 練習7
- 研究2を使って、調査時点ごとのゲイ訪問員による平均効果を推定せよ
- 時間とともにどのように変化していったかを調べよ
- 研究1と研究2から、研究全体の結論を導こう

In [ ]:
temp = data2[data2["study"]==2]
temp = temp.groupby(by=["treatment", "wave"]).mean()[["ssm"]].unstack("treatment")
temp[("ssm", "Same-Sex Marriage Script by Gay Canvasser")] - temp[("ssm", "No Contact")]

研究2でも、1年経っても当事者接触をした方が同性婚支持率を上げる効果が持続していた。

事実として
- ゲイ当事者による訪問は、取り上げる話題が何であれ、同性婚支持を上げやすい
- 同性婚支持を訴える内容での訪問であれば、訪問者が当事者であろうがなかろうが、同性婚支持を上げる事に変わりはないが、1年後も支持をしているかどうかは、訪問者が当事者である方が成績が良い

結論
- 長期的な効果を望む場合は、当事者による訪問をしたほうがよい
- 短期的な効果であれば、ストレートの訪問者でも効果は同等にある

# 自然実験としての指導者暗殺の成功

## 練習1
- いくつの暗殺計画が記録されているか？
- 少なくとも1回は指導者暗殺が計画された国はいくつある？
	- それらの国では年に平均していくつの暗殺計画があるだろうか
 

In [ ]:
data3 = pd.read_csv("officialSource/qss-master/CAUSALITY/leaders.csv")
data3.shape
data3.columns

# 暗殺計画の回数
data3.drop_duplicates(subset=["country", "leadername", "result"]).shape
# 何回も暗殺計画があった人ってだれ？
temp = data3.groupby(by=["country", "leadername", "result"], as_index=False).count()
temp = temp[temp["year"] > 1]
# 同じ人に対して何度も暗殺計画が実行された人はいるが、暗殺計画そのものをカウントする
data3.shape[0]

# 少なくとも1回は指導者暗殺計画が実行された国の数
data3["country"].nunique()

# 暗殺計画があった国の、暗殺計画実行回数の年平均
data3.groupby(by="year").count()["country"].mean()

## 練習2
- 指導者が死亡したら1、生き残ったら0というという「success」カラムを作成して！
- 指導者暗殺計画の成功率はどれくらい？
- 暗殺の成功はランダムに決まるといってよいか？

In [ ]:
# successカラムを作成
# データ状況を把握
data3["result"].unique()
data3["result"].isnull().any()
# successカラムを作成
data3.loc[data3["result"].str.contains("dies"), "success"] = 1
data3.loc[~data3["result"].str.contains("dies"), "success"] = 0

# 暗殺の成功率
data3["success"].mean()

暗殺の成功率は22%。
~~ランダムにきまっているのであれば50%であるはずなので、ランダムにきまっているとは言えず、なんらかの要因が関与していると言える。~~
暗殺の成功がランダムに決まっているかどうかは不明瞭である。少なくとも、全ての暗殺計画が成功しているわけではないという事は言える。しかしながら、他の要因(暗殺者のスキル、リーダーの健康状態、リーダーの警護の状況等)が成功に関与しているのかどうかもわからない状況である。

## 練習3
- 暗殺成功事案と失敗事案、以下の項目の違いはある？
	- 暗殺計画までのポリティ指標の平均
	- 狙われた指導者の年齢
	- 「暗殺の成功はランダムに決まる」という仮定は妥当かどうか？

In [ ]:
# ポリティ指標の平均
data3.groupby(by="success").mean()[["politybefore"]]
data3.groupby(by="success").count()
# ポリティ指標の分布
import matplotlib as mpl
font = {"family":"Ricty"}
mpl.rc('font', **font)
ax = sns.distplot(data3.loc[data3["success"] == 0, "politybefore"].dropna(), label="暗殺失敗")
ax = sns.distplot(data3.loc[data3["success"] == 1, "politybefore"].dropna(), label="暗殺成功", ax=ax)
plt.legend()
plt.show()
# 暗殺成功・失敗別のポリティ指標の分布は正規分布ではない
scipy.stats.shapiro(data3.loc[data3["success"] == 0, "politybefore"].dropna())
scipy.stats.shapiro(data3.loc[data3["success"] == 1, "politybefore"].dropna())

# 狙われた指導者の年齢平均
data3.groupby(by="success").mean()[["age"]]
# 指導者の年齢分布
ax = sns.distplot(data3.loc[data3["success"] == 0, "age"].dropna(), label="暗殺失敗")
ax = sns.distplot(data3.loc[data3["success"] == 1, "age"].dropna(), label="暗殺成功")
plt.legend()
plt.show()
# 年齢の分布が正規分布ではないとは言えない(＝正規分布っぽい)かどうか
scipy.stats.shapiro(data3.loc[data3["success"] == 0, "age"].dropna())
scipy.stats.shapiro(data3.loc[data3["success"] == 1, "age"].dropna())
# 暗殺成功と失敗グループの指導者の年齢に違いがあるといえるか？
scipy.stats.ttest_ind(data3.loc[data3["success"] == 0, "age"].dropna(), data3.loc[data3["success"] == 1, "age"].dropna())

暗殺の成功・失敗別で集計をすると、ポリティ指標からみると、やや民主よりの国の方が暗殺は成功しているように見え、その差は1ポイントほど民主より、となっている。
指導者の年齢でいうと、成功側は平均56歳、失敗側は平均53歳となっており、検定でも有意差ありとなっているので、高年齢であるほど暗殺は成功しやすいという因果はありそう。

## 練習4
- 国が争いの最中にあるかどうかが、暗殺の成功・失敗に影響しているのかどうかを見るために以下のフローで分析してみなはれ
	- 「warbefore」というカラムを追加し、暗殺の前3年間に内戦や国家間戦争の状態にあったかどうかを表す2値を入れる
	- warbefore が暗殺の成功・失敗に影響を与えていたかどうかを調べる

In [ ]:
# 下調べ
1 or 1
1 or 0
1 or None
0 or 0
0 or None
None or None
data3["civilwarbefore"] | data3["interwarbefore"]

data3["civilwarbefore"].isnull().any()
data3["interwarbefore"].isnull().any()

# warbeforeカラム作成
data3.loc[(data3["civilwarbefore"] | data3["interwarbefore"]) == 1, "warbefore"] = 1
data3.loc[(data3["civilwarbefore"] | data3["interwarbefore"]) == 0, "warbefore"] = 0
# 成功失敗別で、warbeforeカラム平均を見る
data3.groupby(by="success").mean()[["warbefore"]]
# 成功失敗別でクロス集計表
pd.crosstab(data3["success"], data3["warbefore"], margins=True)
pd.crosstab(data3["success"], data3["warbefore"], normalize="index", margins=True)
pd.crosstab(data3["success"], data3["warbefore"], normalize="columns", margins=True)
# χ二乗検定
scipy.stats.chi2_contingency(pd.crosstab(data3["success"], data3["warbefore"]))
# 有意差はない

紛争のある無し別で、暗殺の成功・失敗に違いがあるかを見てみた。直近3年以内になんの争いもなかったグループの暗殺成功割合は22%、争いがあったグループの暗殺成功割合は21%で、ほぼ差がない。
χ二乗検定でも有意差がなかったので、暗殺成功と争いがあるかどうかの相関はないと思われる。

## 練習5
指導者の暗殺成功は、民主化を誘引するのか？それとも戦争を誘引するのか？

In [ ]:
# 民主化を誘引するのであれば、暗殺成功国家グループのポリティ指標の前後差分はプラスに偏り、失敗国家はゼロあたりになっている
# 戦争(内戦、国家間)を誘引するのであれば、暗殺成功国家グループは warafter - warbefore が1側に偏り、失敗国家はゼロに偏る、と予想。

# ポリティ指標の差分カラムを作る
data3["polityDiff"] = data3["polityafter"] - data3["politybefore"]

# 暗殺成功・失敗別で、ポリティ指標差分の平均を見てみる
data3.groupby(by="success").mean()[["polityDiff"]]
# 成功国家のポリティ指標の平均は0くらい(＝統治体制に変化なし)
# もともと民主国家だった場合は、暗殺成功したあとでも民主国家のままとなり、変化としては数字に現れないので、確認
diff = data3.groupby(by="success").mean()[["polityDiff"]]
# difference in difference
diff.loc[1] - diff.loc[0]
ax = sns.distplot(data3.loc[data3["success"] == 1, "politybefore"].dropna(), label="暗殺成功")
ax = sns.distplot(data3.loc[data3["success"] == 0, "politybefore"].dropna(), label="暗殺失敗", ax=ax)
plt.legend()
plt.show()
# 暗殺成功国家は、もともと民主国家であった国の方がやや多いので、そもそも民主国家に変化しようがない、とも考えられる。

# 暗殺の成功失敗が、戦争(interwarafter, civilwarafter) によって違うか？
data3["civilwarafter"].isnull().any()
data3["interwarafter"].isnull().any()
data3.loc[(data3["interwarafter"] | data3["civilwarafter"]) == 1, "warafter"] = 1
data3.loc[(data3["interwarafter"] | data3["civilwarafter"]) == 0, "warafter"] = 0

# warDiff を作成
data3["warDiff"] = data3["warafter"] - data3["warbefore"]

diff = data3.groupby(by="success").mean()[["warDiff"]]
# Difference in Difference
diff.loc[1] - diff.loc[0]

ax = sns.distplot(data3.loc[data3["success"] == 1, "warDiff"].dropna(), label="暗殺成功")
ax = sns.distplot(data3.loc[data3["success"] == 0, "warDiff"].dropna(), label="暗殺失敗", ax = ax)
plt.legend()
plt.show()
# クロス集計してみる
pd.crosstab(data3["success"], data3["warDiff"], margins=True)
pd.crosstab(data3["success"], data3["warDiff"], normalize="index", margins=True)
pd.crosstab(data3["success"], data3["warDiff"], normalize="columns", margins=True)
# 暗殺に成功してもしてなくても、争いが起きる割合に大きな差はないっぽい

# χ二乗検定  p=0.4005321355380316, 有意差なし
scipy.stats.chi2_contingency(pd.crosstab(data3["success"], data3["warDiff"]))